#### Downloading & Inspecting EUMETSAT/MSG Data

Code adapted from https://eumetsatspace.atlassian.net/wiki/spaces/EUMDAC/pages/1760198661/Python+Library

Requires signing up for an account with EUMETSAT EO Portal: https://eoportal.eumetsat.int/userMgmt/
Once done, the consumer key and secret can be found under the user information: https://eumetsatspace.atlassian.net/wiki/spaces/DSEV/pages/375652353/API+Authentication

In [ ]:
import autoroot

In [ ]:
import os
import ast
import eumdac
import datetime
import shutil
import pyproj
import rioxarray
import rasterio
import xarray as xr

from pyproj import CRS

In [ ]:
%load_ext autoreload
%autoreload 2

In [8]:
# Insert your personal key and secret into the single quotes
consumer_key = ""
consumer_secret = ""

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

print(f"This token '{token}' expires {token.expiration}")

This token '1cb6fd7f-0cd8-3e2c-bbe4-4f70979bcfa0' expires 2024-02-29 20:06:23.813481


In [9]:
datastore = eumdac.DataStore(token)
datastore.collections

[<class 'eumdac.collection.Collection'>(EO:EUM:DAT:MSG:CLM-IODC),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0082),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0237),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0241),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0274),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0855),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0857),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0584),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0412),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:MSG:MSG15-RSS),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:MULT:HIRSL1),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0836),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:METOP:ASCSZF1B),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:METOP:ASCSZR1B),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0882),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0394),
 <class 'eumdac.collection.C

In [46]:
type(datastore)

eumdac.datastore.DataStore

In [44]:
selected_collection = datastore.get_collection('EO:EUM:DAT:MSG:HRSEVIRI')
selected_collection.search()

<class 'eumdac.collection.SearchResults'>(EO:EUM:DAT:MSG:HRSEVIRI, {'title': None, 'dtend': None, 'publication': None, 'dtstart': None, 'set': None, 'sat': None, 'sort': None, 'geo': None, 'type': None, 'bbox': None})

##### Define download parameters & compile list of products

In [51]:
# Add vertices for polygon, wrapping back to the start point.
geometry = [[-1.0, -1.0],[4.0, -4.0],[8.0, -2.0],[9.0, 2.0],[6.0, 4.0],[1.0, 5.0],[-1.0, -1.0]]

# Set sensing start and end time
# start = datetime.datetime(2021, 10, 10, 0, 0, 0)
# end = datetime.datetime(2021, 10, 10, 23, 59, 59)
start = datetime.datetime.strptime("2021-10-10 12:55:00", "%Y-%m-%d %H:%M:%S")
end = datetime.datetime.strptime("2021-10-10 12:59:00", "%Y-%m-%d %H:%M:%S")

# Retrieve datasets that match our filter
products = selected_collection.search(
    # geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in geometry])),
    dtstart=start,
    dtend=start,
    # dtend=end,
    sort="start,time,1")
    
print(f'Found Datasets: {len(products)} datasets for the given time range')

for product in products:
    print(str(product))

Found Datasets: 1 datasets for the given time range
MSG4-SEVI-MSG15-0100-NA-20211010125743.603000000Z-NA


##### Download products

Each product contains three files (EOPMetadata.xml, manifest.xml, and data.nat files). When downloading the whole product, a .zip folder is created.

In [ ]:
for product in products:
    with product.open() as fsrc, \
            open(fsrc.name, mode='wb') as fdst:
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of product {product} finished.')
print('All downloads are finished.')

In [ ]:
for product in products:    
    print(product.url)

##### Download only selected products & files

In [34]:
selected_product = datastore.get_product(
    product_id='MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA',
    collection_id='EO:EUM:DAT:MSG:HRSEVIRI')

The data is stored in a .nat file

In [55]:
for entry in selected_product.entries:
    print(entry)

EOPMetadata.xml
MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA.nat
manifest.xml


In [ ]:
try:
    with selected_product.open(entry='MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA.nat') as fsrc, \
            open(fsrc.name, mode='wb') as fdst:
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of file {fsrc.name} finished.')
except eumdac.product.ProductError as error:
    print(f"Error related to the product '{selected_product}' while trying to download it: '{error.msg}'")

In [ ]:
try:
    with selected_product.open(entry='EOPMetadata.xml') as fsrc, \
            open(fsrc.name, mode='wb') as fdst:
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of file {fsrc.name} finished.')
except eumdac.product.ProductError as error:
    print(f"Error related to the product '{selected_product}' while trying to download it: '{error.msg}'")

Unfortunately, .nat files are a bit complicated to read/load. One of the best packages is satpy, which then allows loading the files using xarray.

In [ ]:
from satpy import Scene
scn = Scene(reader="seviri_l1b_native", filenames=['MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA.nat'])

In [ ]:
# MSG data contains 12 channels, including the high-resolution visible (HRV) channel.
datasets = scn.available_dataset_names()
datasets

In order to convert the scn object to xarray, each dataset needs to be loaded.

In [ ]:
scn.load(datasets[1:], generate=False)
# Note: The HRV channel comes in 1 km resolution, while the other channels are 3 km resolution.
# Because of the size mismatch, the HRV channel would either need to be downscaled,
# or it needs to the loaded separetely to the other channels


In [ ]:
ds = scn.to_xarray()